In [33]:
import numpy as np
import pandas as pd
import os
from pprint import pprint
import sklearn.pipeline
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import sklearn.neural_network
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.stats import expon, uniform
import string


In [4]:
!ls

README.md          load_train_data.py problem1.py
data_reviews       prob2.ipynb        problem2.py


In [14]:
pd.read_csv("y_train.csv")

,is_positive_sentiment
0,0
1,0
2,0
3,0
4,0
...,...
2395,1
2396,1
2397,1
2398,1


In [18]:
pprint(pd.read_csv("x_train.csv"))
x_amazon = list()
x_imdb = list()
x_yelp = list()

y_data = pd.read_csv("y_train.csv")
# also need to split y data
y_amazon = list()
y_imdb = list()
y_yelp = list()

for i, row in enumerate(pd.read_csv(("x_train.csv")).iterrows()):
    # split data into 3 lists, also get the ith value of y and put in correponding list
    if row[1][0] == "amazon":
        x_amazon.append(row[1][1])
        y_amazon.append(y_data.iloc[i][0])
    elif row[1][0] == "imdb":
        x_imdb.append(row[1][1])
        y_imdb.append(y_data.iloc[i][0])

    elif row[1][0] == "yelp":
        x_yelp.append(row[1][1])
        y_yelp.append(y_data.iloc[i][0])


x_amazon = np.array(x_amazon)
y_amazon = np.array(y_amazon)

     website_name                                               text
0          amazon  Oh and I forgot to also mention the weird colo...
1          amazon                       THAT one didn't work either.
2          amazon                                 Waste of 13 bucks.
3          amazon  Product is useless, since it does not have eno...
4          amazon  None of the three sizes they sent with the hea...
...           ...                                                ...
2395         yelp  The sweet potato fries were very good and seas...
2396         yelp  I could eat their bruschetta all day it is dev...
2397         yelp                               Ambience is perfect.
2398         yelp  We ordered the duck rare and it was pink and t...
2399         yelp       Service was good and the company was better!

[2400 rows x 2 columns]


In [19]:
x_amazon 

array(['Oh and I forgot to also mention the weird color effect it has on your phone.',
       "THAT one didn't work either.", 'Waste of 13 bucks.',
       'Product is useless, since it does not have enough charging current to charge the 2 cellphones I was planning to use it with.',
       'None of the three sizes they sent with the headset would stay in my ears.',
       'Worst customer service.',
       'The Ngage is still lacking in earbuds.',
       'It always cuts out and makes a beep beep beep sound then says signal failed.',
       'the only VERY DISAPPOINTING thing was there was NO SPEAKERPHONE!!!!',
       'Very disappointed in AccessoryOne.',
       'Basically the service was very bad.', 'Bad Choice.',
       'The only thing that disappoint me is the infra red port (irda).',
       'horrible, had to switch 3 times.',
       'It feels poorly constructed, the menus are difficult to navigate, and the buttons are so recessed that it is difficult to push them.',
       "Don't make 

In [35]:
pipeline = sklearn.pipeline.Pipeline([
        # ("bow_feature_extractor", TfidfVectorizer(ngram_range=(1,2), strip_accents="ascii")),
        ("bow_feature_extractor", CountVectorizer(ngram_range=(1,2), strip_accents="ascii")),
        # ("classifier", sklearn.neural_network.MLPClassifier([100, 50, 25],solver="lbfgs", max_iter=2000)),
        ("classifier", sklearn.neural_network.MLPClassifier()),
    ])

In [34]:
distributions = { #  single values just to test on
        # "classifier__hidden_layer_sizes": [(50,50), (500,500), (100, 100, 100, 100, 100), (10, 10, 10, 10, 10)],
        # "classifier__hidden_layer_sizes": [(10), (10, 10), (10, 10, 10), (10, 10, 10, 10), (10, 10, 10, 10, 10, 10), (10, 10, 10, 10, 10, 10, 10), (10, 10, 10, 10, 10, 10, 10, 10)],
        "classifier__hidden_layer_sizes": [(50, 50), (50, 50, 50), (100, 25), (100, 50), (10, 10, 10, 10, 10), ],
        # "classifier__alpha": [0.001],
        "classifier__alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10],
        # "classifier__learning_rate_init": [0.0001, 0.001, 0.01, 0.1, 1],
        "classifier__max_iter": [2000],
        "classifier__solver": [ "lbfgs"],
        # "classifier__solver": ["adam", "lbfgs"],
        # "classifier__learning_rate": ["adaptive", "constant"],
        "classifier__activation": ['relu'],
        # "bow_feature_extractor__min_df": range(0, 5),
        "bow_feature_extractor__min_df": [1],
        "bow_feature_extractor__max_df": [1.0],
    }

In [36]:
num_models = 100000
clf = sklearn.model_selection.RandomizedSearchCV(pipeline, distributions, n_iter = num_models, n_jobs = -1, verbose=2)


In [37]:
clf.fit(x_amazon, y_amazon)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/charlottev/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 30 is smaller than n_iter=100000. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END bow_feature_extractor__max_df=1.0, bow_feature_extractor__min_df=1, classifier__activation=relu, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(50, 50), classifier__max_iter=2000, classifier__solver=lbfgs; total time=   0.9s
[CV] END bow_feature_extractor__max_df=1.0, bow_feature_extractor__min_df=1, classifier__activation=relu, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(50, 50), classifier__max_iter=2000, classifier__solver=lbfgs; total time=   1.0s
[CV] END bow_feature_extractor__max_df=1.0, bow_feature_extractor__min_df=1, classifier__activation=relu, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(50, 50), classifier__max_iter=2000, classifier__solver=lbfgs; total time=   1.0s
[CV] END bow_feature_extractor__max_df=1.0, bow_feature_extractor__min_df=1, classifier__activation=relu, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(50, 50), classifier__max_iter=2000, classifier__solver=lbfgs; total time=   1.1s
[CV] END bow_feature

/Users/charlottev/micromamba/envs/cs135_env/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bow_feature_extractor__max_df=1.0, bow_feature_extractor__min_df=1, classifier__activation=relu, classifier__alpha=0.1, classifier__hidden_layer_sizes=(50, 50), classifier__max_iter=2000, classifier__solver=lbfgs; total time=   3.4s
[CV] END bow_feature_extractor__max_df=1.0, bow_feature_extractor__min_df=1, classifier__activation=relu, classifier__alpha=0.1, classifier__hidden_layer_sizes=(50, 50), classifier__max_iter=2000, classifier__solver=lbfgs; total time=   3.1s
[CV] END bow_feature_extractor__max_df=1.0, bow_feature_extractor__min_df=1, classifier__activation=relu, classifier__alpha=0.1, classifier__hidden_layer_sizes=(50, 50), classifier__max_iter=2000, classifier__solver=lbfgs; total time=   3.9s
[CV] END bow_feature_extractor__max_df=1.0, bow_feature_extractor__min_df=1, classifier__activation=relu, classifier__alpha=0.1, classifier__hidden_layer_sizes=(50, 50), classifier__max_iter=2000, classifier__solver=lbfgs; total time=   3.5s
[CV] END bow_feature_extractor__

RandomizedSearchCV(estimator=Pipeline(steps=[('bow_feature_extractor',
                                              CountVectorizer(ngram_range=(1,
                                                                           2),
                                                              strip_accents='ascii')),
                                             ('classifier', MLPClassifier())]),
                   n_iter=100000, n_jobs=-1,
                   param_distributions={'bow_feature_extractor__max_df': [1.0],
                                        'bow_feature_extractor__min_df': [1],
                                        'classifier__activation': ['relu'],
                                        'classifier__alpha': [0.0001, 0.001,
                                                              0.01, 0.1, 1,
                                                              10],
                                        'classifier__hidden_layer_sizes': [(50,
                                                                            50),
                                                                           (50,
                                                                            50,
                                                                            50),
                                                                           (100,
                                                                            25),
                                                                           (100,
                                                                            50),
                                                                           (10,
                                                                            10,
                                                                            10,
                                                                            10,
                                                                            10)],
                                        'classifier__max_iter': [2000],
                                        'classifier__solver': ['lbfgs']},
                   verbose=2)

In [39]:
results = clf.cv_results_
valid_scores = results['mean_test_score']
param_scores = list(zip(results['params'], valid_scores)) # allows us to map params to scores

# sort scores by BEST at top
param_scores.sort(key=lambda x: x[1], reverse=True)

for params, score in param_scores:
    print(f"Mean Validation Score: {score}, Parameters: {params}")

Mean Validation Score: 0.8262500000000002, Parameters: {'classifier__solver': 'lbfgs', 'classifier__max_iter': 2000, 'classifier__hidden_layer_sizes': (100, 50), 'classifier__alpha': 1, 'classifier__activation': 'relu', 'bow_feature_extractor__min_df': 1, 'bow_feature_extractor__max_df': 1.0}
Mean Validation Score: 0.825, Parameters: {'classifier__solver': 'lbfgs', 'classifier__max_iter': 2000, 'classifier__hidden_layer_sizes': (50, 50), 'classifier__alpha': 1, 'classifier__activation': 'relu', 'bow_feature_extractor__min_df': 1, 'bow_feature_extractor__max_df': 1.0}
Mean Validation Score: 0.825, Parameters: {'classifier__solver': 'lbfgs', 'classifier__max_iter': 2000, 'classifier__hidden_layer_sizes': (50, 50, 50), 'classifier__alpha': 1, 'classifier__activation': 'relu', 'bow_feature_extractor__min_df': 1, 'bow_feature_extractor__max_df': 1.0}
Mean Validation Score: 0.825, Parameters: {'classifier__solver': 'lbfgs', 'classifier__max_iter': 2000, 'classifier__hidden_layer_sizes': (100

In [ ]:
# (50,50, 500, 500, 100, 100, 100, 100, 100) -> 0.78125
# (10, 10, 10, 10, 10, 10, 10, 10, 10, 10) -> 0.6012500000000001 -> Too deep is bad!
# (10, 10, 10, 10, 10) -> 0.7862499999999999